In [19]:
# Step 1: Import Libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display, clear_output
from sklearn.decomposition import PCA
from google.colab import drive, files
import os
from scipy import stats

# Step 2: Global Variable to Store Image
current_image = None

# Step 2: Preprocessing Functions
def compute_mean(image):
    """Calculate the mean of valid pixel values (exclude zeros)."""
    valid_pixels = image[image > 0]
    if valid_pixels.size == 0:
        return "No valid pixels"
    return np.mean(valid_pixels)

def compute_median(image):
    """Calculate the median of valid pixel values (exclude zeros)."""
    valid_pixels = image[image > 0]
    if valid_pixels.size == 0:
        return "No valid pixels"
    return np.median(valid_pixels)

def compute_mode(image):
    """Calculate the mode of valid pixel values (exclude zeros)."""
    valid_pixels = image[image > 0]
    if valid_pixels.size == 0:
        return "No valid pixels"
    mode_result = stats.mode(valid_pixels, keepdims=True)[0][0]
    return mode_result

def compute_svd(image):
    """Perform Singular Value Decomposition and return top 10 singular values."""
    U, s, Vt = np.linalg.svd(image, full_matrices=False)
    return s[:10]

def compute_pca(image):
    """Perform PCA and return explained variance ratio for 1 component."""
    try:
        image_float = image.astype(np.float32)
        flattened = image_float.reshape(-1, 1)
        if flattened.size == 0:
            return "Error: Empty or invalid image for PCA"
        pca = PCA(n_components=1)
        pca.fit(flattened)
        return pca.explained_variance_ratio_[0]
    except Exception as e:
        return f"PCA Error: {str(e)}"

# Step 3: Preprocessing Function
def preprocess_image(image, operations):
    """Apply selected preprocessing operations and return results."""
    results = {}
    for op in operations:
        if op == 'Mean':
            results['Mean'] = compute_mean(image)
        elif op == 'Median':
            results['Median'] = compute_median(image)
        elif op == 'Mode':
            results['Mode'] = compute_mode(image)
        elif op == 'SVD':
            results['SVD'] = compute_svd(image)
        elif op == 'PCA':
            results['PCA'] = compute_pca(image)
    return results

# Step 4: Visualization Function
def plot_results(image, results):
    """Plot original image, SVD singular values, and pixel histogram."""
    plt.figure(figsize=(15, 5))

    # Subplot 1: Original Image
    plt.subplot(1, 3, 1)
    plt.imshow(image, cmap='gray')
    plt.title('Original Depth Image')
    plt.axis('off')

    # Subplot 2: SVD Singular Values (if computed)
    if 'SVD' in results:
        plt.subplot(1, 3, 2)
        plt.plot(results['SVD'], marker='o')
        plt.title('SVD Singular Values')
        plt.xlabel('Component')
        plt.ylabel('Singular Value')

    # Subplot 3: Pixel Intensity Histogram
    plt.subplot(1, 3, 3)
    valid_pixels = image[image > 0]
    if valid_pixels.size > 0:
        plt.hist(valid_pixels.ravel(), bins=50, color='green', alpha=0.7)
    plt.title('Pixel Intensity Histogram')
    plt.xlabel('Intensity')
    plt.ylabel('Frequency')

    plt.tight_layout()
    plt.show()

# Step 4: Save Results Function
def save_results(results, filename='preprocessing_results.txt'):
    """Save preprocessing results to a text file."""
    with open(filename, 'w') as f:
        f.write("Preprocessing Results:\n")
        for key, value in results.items():
            f.write(f"{key}: {value}\n")
    files.download(filename)

# Modified: Load Image Function with DFT/iDFT
def load_image(file_content, display_spectrum=False):
    """Load image, apply DFT/iDFT, and optionally display frequency spectrum."""
    img_array = np.array(Image.open(io.BytesIO(file_content)))
    if len(img_array.shape) == 3:
        img_array = np.mean(img_array, axis=2).astype(np.float32)  # Convert to grayscale
    else:
        img_array = img_array.astype(np.float32)

    # Apply DFT
    dft = np.fft.fft2(img_array)
    # Shift zero frequency to center for visualization
    dft_shift = np.fft.fftshift(dft)

    # Display magnitude spectrum if requested
    if display_spectrum:
        magnitude_spectrum = 20 * np.log(np.abs(dft_shift) + 1e-10)  # Log scale
        plt.figure(figsize=(6, 6))
        plt.imshow(magnitude_spectrum, cmap='gray')
        plt.title('DFT Magnitude Spectrum')
        plt.axis('off')
        plt.show()

    # Apply iDFT to reconstruct image
    idft_shift = np.fft.ifftshift(dft_shift)
    reconstructed = np.fft.ifft2(idft_shift)
    # Convert back to real-valued image, clip to [0, 255], and cast to uint8
    reconstructed = np.clip(np.real(reconstructed), 0, 255).astype(np.uint8)

    return reconstructed

# Modified: Load Image from Dataset Function with DFT/iDFT
def load_image_from_path(filepath, display_spectrum=False):
    """Load image from a file path with DFT/iDFT."""
    if not os.path.exists(filepath):
        return None
    img_array = np.array(Image.open(filepath))
    if len(img_array.shape) == 3:
        img_array = np.mean(img_array, axis=2).astype(np.float32)
    else:
        img_array = img_array.astype(np.float32)

    # Apply DFT
    dft = np.fft.fft2(img_array)
    dft_shift = np.fft.fftshift(dft)

    # Display magnitude spectrum if requested
    if display_spectrum:
        magnitude_spectrum = 20 * np.log(np.abs(dft_shift) + 1e-10)
        plt.figure(figsize=(6, 6))
        plt.imshow(magnitude_spectrum, cmap='gray')
        plt.title('DFT Magnitude Spectrum')
        plt.axis('off')
        plt.show()

    # Apply iDFT
    idft_shift = np.fft.ifftshift(dft_shift)
    reconstructed = np.fft.ifft2(idft_shift)
    reconstructed = np.clip(np.real(reconstructed), 0, 255).astype(np.uint8)

    return reconstructed

# Step 1: File Upload Widget
upload_button = widgets.FileUpload(
    accept='.png,.jpg,.jpeg',
    multiple=False,
    description='Upload Image'
)

# Step 4: Dataset Path Input Widget
dataset_path_input = widgets.Text(
    value='/content/drive/My Drive/kitti_depth_images',
    description='Dataset Path:',
    layout={'width': '500px'}
)

# Step 4: Load from Dataset Button
load_dataset_button = widgets.Button(
    description='Load from Dataset',
    button_style='info',
    tooltip='Load image from dataset path'
)

# Step 2: Preprocessing Options Widget
preprocess_options = widgets.SelectMultiple(
    options=['Mean', 'Median', 'Mode', 'SVD', 'PCA'],
    value=['Mean'],
    description='Operations',
    disabled=False
)

# Step 3: Process Button Widget
process_button = widgets.Button(
    description='Process Image',
    button_style='success',
    tooltip='Click to process the image'
)

# Step 4: Save Results Button
save_button = widgets.Button(
    description='Save Results',
    button_style='primary',
    tooltip='Save results to a text file'
)

# New: Checkbox for DFT Spectrum Display
show_dft_spectrum = widgets.Checkbox(
    value=True,
    description='Show DFT Spectrum',
    disabled=False
)

# Step 1: Output Widget
output = widgets.Output()

# Modified: Upload Callback
def on_upload_change(change):
    """Handle file upload and display the image."""
    global current_image
    with output:
        clear_output()
        if upload_button.value:
            file_info = list(upload_button.value.values())[0]
            file_content = file_info['content']
            # Pass show_dft_spectrum value
            current_image = load_image(file_content, display_spectrum=show_dft_spectrum.value)
            plt.figure(figsize=(6, 6))
            plt.imshow(current_image, cmap='gray')
            plt.title('Reconstructed Depth Image (Post-DFT/iDFT)')
            plt.axis('off')
            plt.show()

# Modified: Load Dataset Callback
def on_load_dataset_clicked(b):
    """Handle loading image from dataset path."""
    global current_image
    with output:
        clear_output()
        dataset_path = dataset_path_input.value
        try:
            drive.mount('/content/drive', force_remount=True)
            image_files = [f for f in os.listdir(dataset_path) if f.endswith(('.png', '.jpg', '.jpeg'))]
            if not image_files:
                print("No images found in the specified directory!")
                return
            image_path = os.path.join(dataset_path, image_files[0])
            current_image = load_image_from_path(image_path, display_spectrum=show_dft_spectrum.value)
            if current_image is None:
                print(f"Failed to load image: {image_path}")
                return
            plt.figure(figsize=(6, 6))
            plt.imshow(current_image, cmap='gray')
            plt.title(f'Reconstructed Image: {image_files[0]} (Post-DFT/iDFT)')
            plt.axis('off')
            plt.show()
        except Exception as e:
            print(f"Error loading dataset: {str(e)}")

# Step 3: Process Button Callback
def on_process_clicked(b):
    """Handle process button click and display results."""
    with output:
        clear_output()
        if current_image is None:
            print("Please upload or load an image first!")
            return
        selected_ops = preprocess_options.value
        if not selected_ops:
            print("Please select at least one preprocessing operation!")
            return

        # Process the image
        results = preprocess_image(current_image, selected_ops)

        # Display visualizations
        plot_results(current_image, results)

        # Display results
        print("Preprocessing Results:")
        for key, value in results.items():
            print(f"{key}: {value}")

# Step 4: Save Button Callback
def on_save_clicked(b):
    """Handle save button click."""
    with output:
        if current_image is None:
            print("Please upload an image!")
            return
        selected_ops = preprocess_options.value
        if not selected_ops:
            print("Please select at least one preprocessing operation!")
            return
        results = preprocess_image(current_image, selected_ops)
        save_results(results)

# Step 1: Assign Upload Callback
upload_button.observe(on_upload_change, names='value')

# Step 3: Assign Process Button Callback
process_button.on_click(on_process_clicked)

# Step 4: Assign Load and Save Callbacks
load_dataset_button.on_click(on_load_dataset_clicked)
save_button.on_click(on_save_clicked)

# Step 4: Display Interface
display(widgets.VBox([
    widgets.Label('KITTI 2D Depth Image Processor'),
    widgets.HBox([upload_button, load_dataset_button]),
    dataset_path_input,
    show_dft_spectrum,  # New: Checkbox for DFT spectrum
    preprocess_options,
    widgets.HBox([process_button, save_button]),
    output
]))

# Step 4: Instructions
print("Instructions:")
print("1. Upload an image using 'Upload Image' or load from dataset using 'Load from Dataset'.")
print("2. For dataset, enter the path (e.g., '/content/drive/My Drive/kitti_depth_images').")
print("3. Check 'Show DFT Spectrum' to display the frequency spectrum during loading.")
print("4. Select one or more preprocessing operations (hold Ctrl for multiple).")
print("5. Click 'Process Image' to compute and visualize results.")
print("6. Click 'Save Results' to download results as a text file.")

Instructions:
1. Upload an image using 'Upload Image' or load from dataset using 'Load from Dataset'.
2. For dataset, enter the path (e.g., '/content/drive/My Drive/kitti_depth_images').
3. Check 'Show DFT Spectrum' to display the frequency spectrum during loading.
4. Select one or more preprocessing operations (hold Ctrl for multiple).
5. Click 'Process Image' to compute and visualize results.
6. Click 'Save Results' to download results as a text file.


In [5]:
!pip install gradio


In [ ]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.decomposition import PCA
from scipy import stats
import io

def compute_mean(image):
    valid = image[image > 0]
    return float(np.mean(valid)) if valid.size else "No valid pixels"

def compute_median(image):
    valid = image[image > 0]
    return float(np.median(valid)) if valid.size else "No valid pixels"

def compute_mode(image):
    valid = image[image > 0]
    return float(stats.mode(valid, keepdims=True)[0][0]) if valid.size else "No valid pixels"

def compute_svd(image):
    _, s, _ = np.linalg.svd(image, full_matrices=False)
    return s[:10]

def compute_pca(image):
    try:
        flat = image.reshape(-1, 1)
        pca = PCA(n_components=1)
        pca.fit(flat)
        return float(pca.explained_variance_ratio_[0])
    except:
        return "PCA Error"

def dft_idft(pil_img):
    img_gray = pil_img.convert("L")
    img_array = np.array(img_gray).astype(np.float32)
    dft = np.fft.fftshift(np.fft.fft2(img_array))
    idft = np.fft.ifft2(np.fft.ifftshift(dft))
    return np.clip(np.real(idft), 0, 255).astype(np.uint8), dft

def fig_to_image(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png")
    buf.seek(0)
    plt.close(fig)
    return Image.open(buf)

def process_image(image, operations, show_spectrum):
    if image is None:
        return None, "No image uploaded", None

    reconstructed, dft = dft_idft(image)

    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(reconstructed, cmap='gray')
    axs[0].set_title("Reconstructed Image")
    axs[0].axis('off')
    valid = reconstructed[reconstructed > 0]
    axs[1].hist(valid.ravel(), bins=50, color='green')
    axs[1].set_title("Histogram")
    img_plot = fig_to_image(fig)

    spectrum_img = None
    if show_spectrum:
        mag = 20 * np.log(np.abs(dft) + 1e-10)
        fig_spec, ax = plt.subplots()
        ax.imshow(mag, cmap='gray')
        ax.axis('off')
        ax.set_title("DFT Spectrum")
        spectrum_img = fig_to_image(fig_spec)

    results = {}
    for op in operations:
        if op == "Mean":
            results["Mean"] = compute_mean(reconstructed)
        elif op == "Median":
            results["Median"] = compute_median(reconstructed)
        elif op == "Mode":
            results["Mode"] = compute_mode(reconstructed)
        elif op == "SVD":
            results["SVD"] = compute_svd(reconstructed)
        elif op == "PCA":
            results["PCA"] = compute_pca(reconstructed)

    result_text = ""
    for k, v in results.items():
        if isinstance(v, (list, np.ndarray)):
            v = ", ".join(f"{x:.2f}" for x in v)
        result_text += f"{k}: {v}\n"

    return img_plot, result_text.strip(), spectrum_img

# ----------- Gradio App -----------
with gr.Blocks() as app:
    gr.Markdown("## 📡 LiDAR Image Processor")

    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            img_input = gr.Image(type='pil', label="Upload Image")
            show_spectrum = gr.Checkbox(label="Show DFT Spectrum", value=True)
        with gr.Column(scale=1):
            img_plot = gr.Image(label="Image + Histogram")

    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            operations = gr.CheckboxGroup(
                ["Mean", "Median", "Mode", "SVD", "PCA"],
                value=["Mean"],
                label="Preprocessing Options"
            )
        with gr.Column(scale=1):
            result_output = gr.Textbox(label="Output Results", lines=6)
            spectrum_output = gr.Image(label="DFT Spectrum (if selected)")

    run_button = gr.Button("🚀 Process Image")
    run_button.click(
        fn=process_image,
        inputs=[img_input, operations, show_spectrum],
        outputs=[img_plot, result_output, spectrum_output]
    )

app.launch(share=True)
